In [ ]:
import os, sys
import pandas as pd
import numpy as np
import math
import glob
from scipy import stats
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import subprocess
import json
import itertools
import re

from IPython.display import display, HTML

%matplotlib notebook

cc_algorithms = ['lia', 'olia', 'balia', 'wvegas', 'cubic']

In [ ]:
def rename_all():
    folder = './logs/'
    for _ in range(8):
        for dirpath, dirnames, filenames in os.walk(folder):
            for d in filenames + dirnames:
                if '-' in d:
                    full_path = '{}/{}'.format(dirpath, d)
                    os.rename(full_path, full_path.replace('-', '_'))
rename_all()

In [ ]:
# Reading JSON config files
def load_config(topology):
    file_name = './topologies/{}.json'.format(topology)
    if not os.path.isfile(file_name):
        print('JSON topology file not found! {}'.format(file_name))
        return None

    with open(file_name, 'r') as f:
        config = json.load(f)

    return config


def get_iperf_pairings(topology):
    config = load_config(topology)
    pairs = []
    for node in [node for node in config['nodes'] if node['id'].startswith('h')]:
        if 'server' in node['properties']:
            pairs.append((str(node['id']), str(node['properties']['server'])))

    # make sure every host is included in some connection
    hosts = itertools.chain.from_iterable(pairs)
    for node in [node for node in config['nodes'] if node['id'].startswith('h')]:
        if node['id'] not in hosts:
            print('Host {} not contained in any host pairings!'.format(node))
    return pairs


def num_clients(topology):
    pairs = get_iperf_pairings(topology)
    return len(pairs)


def confidence_interval(series, z=1.96):
    """
    Calculate confidence interval for a given series. Default is 95% confidence interval.
    See https://en.wikipedia.org/wiki/Confidence_interval#Basic_steps for further values.
    """
    stats = series.agg(['mean', 'count', 'std'])
    return z*stats['std']/math.sqrt(stats['count'])

In [ ]:
# TODO simplify and use in upper function to eliminate duplicat code
def read_iperf_srv_tp_trace(folder, client, server, repetitions):
    file_name = folder + '/{}_{}_iperf.csv'
    
    tp_dfs = pd.DataFrame()
    for rep in range(repetitions):
        ser_file = file_name.format(rep, server)
            
        with open(ser_file, 'r') as f:
            content = f.read().replace('iperf3: interrupt - the server has terminated', '').strip().splitlines()
            content = content[6:-4] # TODO replace with conditional cutting
            df = pd.DataFrame([l.replace('[', '').replace(']','').split() for l in content],
                             columns=['ID', 'Interval', 'Interval_unit', 'Transfer', 'Transfer_unit', 'Bandwidth', 'Bandwidth_unit'])
            df['repetition'] = rep
            tp_dfs = tp_dfs.append(df, ignore_index=True)
            
    tp_dfs['client'] = client
    tp_dfs['server'] = server
    tp_dfs['Interval_start'] = tp_dfs['Interval'].str.split('-').str[0].astype(float)
    tp_dfs['Interval_end'] = tp_dfs['Interval'].str.split('-').str[1].astype(float)
    tp_dfs['ID'] = pd.to_numeric(tp_dfs['ID'])
    tp_dfs['Transfer'] = pd.to_numeric(tp_dfs['Transfer'])
    tp_dfs['Bandwidth'] = pd.to_numeric(tp_dfs['Bandwidth'])
    return tp_dfs

    
def read_pcap_csv(file_name):
    df = pd.read_csv(file_name, delimiter='\t')
    # Note: puts rtt into [ms] instead of [s]
    df['tcp.analysis.ack_rtt'] = df['tcp.analysis.ack_rtt'] * 1000
    return df


def read_single_iperf_experiment_rtt(topology, ccs, rates, delays, store_info=True):
    folder = './logs/{}/{}/{}/{}/'.format(topology, ccs, rates, delays)
    pairs = get_iperf_pairings(topology)
    cli_to_srv = {c: s for c, s in pairs}
    
    df = pd.DataFrame()
    if not os.path.exists(folder):
        print('Attention, folder {} does not exist!'.format(folder))
    
    for dirpath, dirnames, filenames in os.walk(folder):
        client_tuples = [(name,) + extract_rep_host_name(name) for name in filenames]
        client_tuples = [t for t in client_tuples if t[0].endswith('iperf_dump.csv')]
        
        # add each client as a rtt datapoint
        for file_name, host_name, rep in client_tuples:
            pcap = read_pcap_csv('{}/{}'.format(dirpath, file_name))
            pcap['repetition'] = rep
            pcap['host'] = host_name
            df = df.append(pcap, ignore_index=True)
            
    df = df.dropna(subset=['tcp.analysis.ack_rtt'])
    if store_info:
        df['topology'] = topology
        df['ccs'] = ccs
        df['bw'] = rates
        df['delays'] = delays
    return df


def extract_rep_host_name(filename):
    """ Returns a tuple (host_name, rep) """
    rep_hname = filename.split('_')
    return rep_hname[1], int(rep_hname[0])


def read_single_iperf_goodput(folder, client_file):
    with open('{}/{}'.format(folder, client_file), 'r') as f:
        content = f.read()[-300:] \
                    .replace('iperf Done.', '') \
                    .replace('iperf3: interrupt - the server has terminated', '') \
                    .strip().splitlines()
        recv_line = content[-1].split()
        assert 'receiver' in recv_line
        return float(recv_line[6])


def load_single_iperf_experiment_tp(topology, ccs, rates, delays, store_info=True):
    folder = './logs/{}/{}/{}/{}/'.format(topology, ccs, rates, delays)
    pairs = get_iperf_pairings(topology)
    cli_to_srv = {c: s for c, s in pairs}
    
    if not os.path.exists(folder):
        print('Attention, folder {} does not exist!'.format(folder))
        
    columns = {}
    for srv in cli_to_srv.values():
        columns[srv + '_tp'] = {}
    
    for dirpath, dirnames, filenames in os.walk(folder):
        client_tuples = [(name,) + extract_rep_host_name(name) for name in filenames]
        client_tuples = [t for t in client_tuples if t[0].endswith('iperf.csv') and t[1] in cli_to_srv.keys()]
        
        # add each client as a tp datapoint
        for file_name, host_name, rep in client_tuples:
            tp = read_single_iperf_goodput(dirpath, file_name)
            srv_name = cli_to_srv[host_name]
            columns[srv_name + '_tp'][rep] = tp
    df = pd.DataFrame(columns)
    df.index.name = 'rep'
    if store_info:
        df['topology'] = topology
        df['ccs'] = ccs
        df['bws'] = rates
        df['delays'] = delays
    return df


def load_iperf_experiments_new(topology, repetitions=3):
    """
    Read in log data from experiments, every throughput should come out in [Mbps] and times in [ms].
    One exception to this rule is the relative time since experiment start in the rtt trace.
    """
    df = pd.DataFrame()
    pairs = get_iperf_pairings(topology)

    for dirpath, dirnames, filenames in os.walk('./logs/{}'.format(topology)):
        if dirnames:
            continue
        
        # Leaf folder, read in files and analyze
        # dirpath has form "./logs/two_paths/cubic/25Mbps-9Mbps/10ms-10ms"
        dirpath_split = dirpath.split('/') # ['.', 'logs', 'two_paths', 'lia', '9Mbps-13Mbps', '10ms-10ms']
        cc_name = dirpath_split[-3]
        cc = cc_name
        bw_name = dirpath_split[-2]
        bws = bw_name.split('_')
        de_name = dirpath_split[-1]
        des = de_name.split('_')
        
        df_run = load_single_iperf_experiment_tp(topology, cc_name, bw_name, de_name, store_info=False)
        df_run_rtt = read_single_iperf_experiment_rtt(topology, cc_name, bw_name, de_name, store_info=False)
        
        df_run_rtt = df_run_rtt.groupby(['repetition', 'host'])['tcp.analysis.ack_rtt'].mean()

        # read in data for pairings
        row = {'cc': cc}
        for cli, ser in pairs:
            srv_tp_col = df_run[ser + '_tp']
            row[ser + '_tp'] = srv_tp_col.mean()
            row[ser + '_tp_conf'] = confidence_interval(srv_tp_col)
            
            cli_rtt_col = df_run_rtt[:, cli]
            row[cli + '_rtt'] = cli_rtt_col.mean()
            row[cli + '_rtt_conf'] = confidence_interval(cli_rtt_col)

        # Add bw and de groups to dfs
        for bandwidth, group in zip(bws, ['a', 'b', 'c', 'd']):
            b = int(re.sub('[^0-9]', '', bandwidth))
            row['bw_' + group] = b
        for delay, group in zip(des, ['a', 'b', 'c', 'd']):
            d = float(re.sub('[^0-9.]', '', delay))
            row['de_' + group] = d
        df = df.append(row, ignore_index=True)

    return df


# load_iperf_experiments_new('shared_link')
# load_single_iperf_experiment_tp('single_bottleneck', 'lia-lia', '10Mbps', '10.0ms')
# read_single_iperf_experiment_rtt('two_paths', 'lia', '10Mbps-10Mbps', '10.0ms-10.0ms').head(10)

In [ ]:
def load_goodputs_series(folder, filenames):
    folder = './logs/{}/{}/{}/{}/'.format(topology, ccs, rates, delays)
    pairs = get_iperf_pairings(topology)
    cli_to_srv = {c: s for c, s in pairs}
    
    if not os.path.exists(folder) or len(filenames) == 0:
        print('Attention, folder {} does not exist or no files are therein!'.format(folder))
        
    columns = {}
    for srv in cli_to_srv.values():
        columns[srv + '_tp'] = {}
    
    for dirpath, dirnames, filenames in os.walk(folder):
        print(dirpath)
        print(filenames[0])
        return
        client_tuples = [(name,) + extract_rep_host_name(name) for name in filenames]
        client_tuples = [t for t in client_tuples if t[0].endswith('iperf.csv') and t[1] in cli_to_srv.keys()]
        
        # add each client as a tp datapoint
        for file_name, host_name, rep in client_tuples:
            tp = read_single_iperf_goodput(dirpath, file_name)
            srv_name = cli_to_srv[host_name]
            columns[srv_name + '_tp'][rep] = tp
    df = pd.DataFrame(columns)
    df.index.name = 'rep'
    if store_info:
        df['topology'] = topology
        df['ccs'] = ccs
        df['bws'] = rates
        df['delays'] = delays
    return df

def load_single_iperf_experiment_tp(topology, ccs, rates, delays, store_info=True):
    folder = './logs/{}/{}/{}/{}/'.format(topology, ccs, rates, delays)
    pairs = get_iperf_pairings(topology)
    cli_to_srv = {c: s for c, s in pairs}
    
    
def read_single_iperf_output(folder, host_file, include_info=True):
    with open('{}/{}'.format(folder, host_file), 'r') as f:
        content = f.read() \
                        .replace('iperf Done.', '') \
                        .replace('[', '') \
                        .replace(']', '') \
                        .replace('iperf3: interrupt - the server has terminated', '') \
                        .strip().splitlines()
        
        start_i, end_i = 999, 999
        for i in range(10):
            if content[i].strip().startswith('ID'):
                start_i = i + 1
            if content[-i].strip().startswith('ID'):
                end_i = -(i + 1)
                
        if start_i > 100 or end_i > 100:
            print('Attention, file seems to be of incorrect form: {}/{}'.format(folder, client_file))
            
        content = content[start_i:end_i]
        column_names = ['ID', 'Interval', 'Interval_unit', 'Transfer', 'Transfer_unit',
                        'Bandwidth', 'Bandwidth_unit', 'Retr', 'Cwnd', 'Cwnd_unit']
        if len(content[0].split()) < 10:
            column_names = column_names[:-3]
        
        df = pd.DataFrame([l.split() for l in content],
                          columns=column_names)
        name_split = host_file.split('_')
        if include_info:
            df['repetition'] = name_split[0]
            df['host'] = name_split[1]
            df['Interval_start'] = df['Interval'].str.split('-').str[0].astype(float)
            df['Interval_end'] = df['Interval'].str.split('-').str[1].astype(float)
            df['ID'] = pd.to_numeric(df['ID'])
            df['Transfer'] = pd.to_numeric(df['Transfer'])
            df['Bandwidth'] = pd.to_numeric(df['Bandwidth'])
        return df

# read_iperf_srv_tp_trace_new('./logs/two_paths/lia/10Mbps-10Mbps/10.0ms-10.0ms/', 'h1', 'h2')
# load_single_iperf_experiment_tp_new('two_paths', 'lia', '10Mbps-10Mbps', '10.0ms-10.0ms')
# read_iperf_output('./logs/two_paths/lia/10Mbps-10Mbps/10.0ms-10.0ms/', '0-h2_iperf.csv')
read_single_iperf_output('./logs/two_paths/lia/10Mbps_10Mbps/30.0ms_30.0ms/', '0_h2_iperf.csv')
# read_single_iperf_goodput('./logs/two_paths/lia/10Mbps_10Mbps/10.0ms_10.0ms/', '0_h2_iperf.csv')

In [ ]:
def init_plots(df, delay_point=30, bw_point=10):
    if len([c for c in df.columns if c.startswith('de_')]) > 2:
        print('Plotting does not yet support more than two delay groups!')
        return
    
    n_clients = len([c for c in df.columns if c.endswith('_rtt_conf')])
    fig = plt.figure(figsize=(9.5, 3.5*n_clients))
    
    # TODO handle this better, this only takes out part of what should be removed
    for column in [c for c in df.columns if c.startswith('bw')]:
        df = df[df[column] == bw_point]

    for i in range(n_clients):
        ax1 = fig.add_subplot(n_clients, 2, 2*i+1)
        ax2 = fig.add_subplot(n_clients, 2, 2*i+2)
        cli = 'h{}'.format(i*2+1)
        serv = 'h{}'.format(i*2+2)
        
        for cc in df['cc'].unique():
            tmp = df[df['cc'] == cc]
            if 'de_b' in tmp.columns:
                tmp = tmp[tmp['de_b'] == delay_point]
            
            tmp = tmp.sort_values('de_a')

            a1 = tmp.plot(x='de_a', y='{}_tp'.format(serv), yerr='{}_tp_conf'.format(serv), label=cc,
                          ax=ax1, grid=True) #, ylim=(0,22))
            a2 = tmp.plot(x='de_a', y='{}_rtt'.format(cli), yerr='{}_rtt_conf'.format(cli), label=cc,
                          ax=ax2, grid=True) #, ylim=(0,130))

        ax1.set_title('Iperf Throughput on {}'.format(serv))
        ax1.set_ylabel('Mbps Throughput')
        ax1.set_xlabel('ms delay of (second) link')
        ax1.autoscale(True, axis='x')

        ax2.set_title('Iperf Packet RTT on {}'.format(cli))
        ax2.set_ylabel('ms')
        ax2.set_xlabel('ms delay of (second) link')
        ax2.autoscale(True, axis='x')

        if 'de_b' in df.columns:
            ax1.axvline(delay_point, color='black', ls=':')
            ax2.axvline(delay_point, color='black', ls=':')
    
    plt.tight_layout()
        

def de_3d_plot(df):
    fig = plt.figure(figsize=(9.5, 9.5))
    for i, cc in enumerate(df['cc'].unique()):
        ax = fig.add_subplot(3, 2, i+1, projection='3d')
        ax.set_title('Throughput for {}'.format(cc))
        ax.set_xlabel('ms delay group a')
        ax.set_ylabel('ms delay group b')

        tmp = df[df['cc'] == cc]
        tmp = tmp[(tmp['bw_a'] == 10) & (tmp['bw_b'] == 10)]
        ax.plot_trisurf(tmp['de_a'], tmp['de_b'], tmp['h2_tp'], cmap='viridis');
        ax.set_zlim(0, 25)
        ax.view_init(25, 25)
    plt.tight_layout()
        
def tp_3d_plot(df):
    for cc in df['cc'].unique():
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.set_title('Throughput for {}'.format(cc))
        ax.set_xlabel('ms delay group a')
        ax.set_ylabel('ms delay group b')

        tmp = df[df['cc'] == cc]
        tmp = tmp[(tmp['bw_a'] == 10) & (tmp['bw_b'] == 10)]
        ax.plot_trisurf(tmp['de_a'], tmp['de_b'], tmp['h2_tp'], cmap='viridis');
        ax.view_init(25, 25)
    plt.tight_layout()
    
# init_plots(two_paths_df)
# de_3d_plot(two_paths_df)

In [ ]:
def plot_rtt_timeline_per_cc(topo, bw_dir, de_dir, client='h1', smoothing=3):
    fig = plt.figure(figsize=(9.5, 9.5))
    subplots = []
    max_y = 0
    num_lines = 0
    for i, cc in enumerate(cc_algorithms):
        cc = '_'.join([cc] * num_clients(topo))
        ax = fig.add_subplot(3, 2, i+1)
        subplots.append(ax)
        df = read_single_iperf_experiment_rtt(topo, cc, bw_dir, de_dir)
        df = df[df['tcp.analysis.ack_rtt'] > 0.06].sort_values('frame.time_relative')
        max_y = max(max_y, df['tcp.analysis.ack_rtt'].max())
        
        for key, grp in df.groupby(['repetition', 'tcp.stream']):
            num_lines += 1
            if smoothing > 1:
                ax.plot(grp['frame.time_relative'], grp['tcp.analysis.ack_rtt'].rolling(window=smoothing).mean(),
                        label='rep/stream {}'.format(key))
            else:
                sub = grp.plot(ax=ax, style='+', x='frame.time_relative', y='tcp.analysis.ack_rtt',
                               label='rep/stream {}'.format(key))
            
        ax.set_title('{}: RTT Timeline on Host {}'.format(cc, client))
        ax.set_ylabel('ms RTT')
        ax.set_xlabel('s Experiment Time')
        ax.autoscale(True)
        ax.grid()
        ax.legend()
        if num_lines > 6:
            ax.legend().remove()
    
    for ax in subplots:
        ax.set_ylim(0, max_y)
    plt.tight_layout()
    
def autolabel(rects, ax):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for i, rect in enumerate(rects):
        height = rect.get_height()
        ax.annotate('{:.2f}'.format(height), xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3 + i*2),  # 3 points vertical offset
                    textcoords="offset points", ha='center', va='bottom', rotation=90)
        
def plot_tp_timeline_per_cc(topo, bw_dir, de_dir, servers=['h2'], repetitions=3, smoothing=1, y_max=35):
    fig = plt.figure(figsize=(9.5, 9.5*len(servers)))
    for i, cc in enumerate(cc_algorithms):
        cc = '_'.join([cc] * num_clients(topo))
        for j, serv in enumerate(servers):
            ax = fig.add_subplot(3*len(servers), 2, len(servers)*i+j+1)
            folder = './logs/{}/{}/{}/{}'.format(topo, cc, bw_dir, de_dir)
            df = read_iperf_srv_tp_trace(folder, client='h1', server=serv, repetitions=repetitions)

            for key, grp in df.groupby(['repetition']):
                width = grp['Interval_start'].max() / 25
                if smoothing > 1:
                    ax.plot(grp['Interval_start'], grp['Bandwidth'].rolling(window=smoothing).mean(),
                            label='rep {}'.format(key))
                else:
                    grp.plot(ax=ax, x='Interval_start', y='Bandwidth', style='+', label='rep {}'.format(key))
                b = ax.bar(grp['Interval_start'].max()+(key+1)*width+5, grp['Bandwidth'].mean(), width=width)
                ax.autoscale(True)
                autolabel(b, ax)

            ax.title.set_text('{}: Throughput Timeline on Host {}'.format(cc, serv))
            ax.set_ylabel('Mbps Throughput')
            ax.set_xlabel('s Experiment Time')
            ax.set_ylim((0, y_max))
            ax.legend()
    plt.tight_layout()

# plot_rtt_timeline_per_cc('shared_link', '15Mbps', '10ms')
# plot_tp_timeline_per_cc('shared_link', '15Mbps', '10ms', smoothing=10)

# Two Paths

In [ ]:
two_paths_df = load_iperf_experiments_new('two_paths')

In [ ]:
two_paths_df.head(2)

In [ ]:
# two_paths_df.drop(['bw_a', 'bw_b', 'de_a', 'de_b'], axis=1).groupby('cc').describe()
two_paths_df.groupby('cc')['h2_tp'].describe()

In [ ]:
init_plots(two_paths_df, 30)
# init_plots(two_paths_df, 90)
de_3d_plot(two_paths_df)

In [ ]:
plot_rtt_timeline_per_cc('two_paths', '10Mbps_10Mbps', '30.0ms_30.0ms', 'h1', smoothing=10)
plot_tp_timeline_per_cc('two_paths', '10Mbps_10Mbps', '30.0ms_30.0ms', ['h2'], 3, smoothing=5)

In [ ]:
fig = plt.figure(figsize=(9.5, 9.5))
big_df = pd.DataFrame()

for i, cc in enumerate(cc_algorithms):
    df = load_single_iperf_experiment_tp('two_paths', cc, '10Mbps_10Mbps', '30.0ms_30.0ms')
    df['bw'] = 20
    df['de'] = 10
    df['lost tp'] = (df['bw'] - df['h2_tp'])
    big_df = big_df.append(df, ignore_index=True)
    
    ax = fig.add_subplot(3, 2, i+1)
    df['lost tp'].hist(ax=ax, cumulative=True, density=1, bins=50)
    ax.title.set_text('{}: CDF wasted bw'.format(cc))
    ax.set_ylabel('CDF')
    ax.set_xlabel('absolute bw wasted')
plt.tight_layout()
plt.show()
big_df

# Shared Link

In [ ]:
shared_link_df = load_iperf_experiments_new('shared_link')

In [ ]:
print('BW: ', shared_link_df['bw_a'].unique())
print('De: ', shared_link_df['de_a'].unique())
# shared_link_df = shared_link_df.drop_duplicates(subset=['bw_a', 'cc', 'de_a'])
shared_link_df

In [ ]:
tmp_shared_link_df = shared_link_df # shared_link_df[shared_link_df['de_a'] != 10]
init_plots(tmp_shared_link_df)
# de_3d_plot(shared_link_df)

In [ ]:
# weird results for 0ms/30ms
bw, de = '10Mbps', '30.0ms'
plot_rtt_timeline_per_cc('shared_link', bw, de, 'h1', 1)
plot_tp_timeline_per_cc('shared_link', bw, de, ['h2', 'h4'], 3, smoothing=5, y_max=15)

In [ ]:
fig = plt.figure(figsize=(9.5, 9.5))
dfs = {}

for i, cc in enumerate(cc_algorithms):
    ccs = '_'.join([cc]*2)
    df = load_single_iperf_experiment_tp('shared_link', ccs, '10Mbps', '30.0ms')
    df['bw'] = 10
    df['de'] = 10
    df['lost tp'] = (df['bw'] - df['h2_tp'] - df['h4_tp']) / df['bw']
    df['tp_diff'] = (df['h2_tp'] - df['h4_tp']).abs()

    ax = fig.add_subplot(3, 2, i+1)
    df['lost tp'].hist(ax=ax, cumulative=True, density=1, bins=50)
    ax.title.set_text('{}: CDF wasted bw'.format(cc))
    ax.set_ylabel('CDF')
    ax.set_xlabel('absolute bw wasted')
    dfs[cc] = df
plt.tight_layout()
dfs['wvegas']

# MP vs SP

In [ ]:
mp_vs_sp_df = load_iperf_experiments_new('mp_vs_sp')

In [ ]:
mp_vs_sp_df['h2_tp_on_b'] = mp_vs_sp_df['h2_tp'] - mp_vs_sp_df['bw_a']
# mp_vs_sp_df[['cc', 'bw_a', 'bw_b', 'h2_tp_on_b']]
mp_vs_sp_df.head(10)

In [ ]:
fig = plt.figure(figsize=(9.5, 9.5))
cur_df = mp_vs_sp_df[mp_vs_sp_df['bw_a'] == 15]
cur_df['x'] = cur_df['bw_a']/cur_df['bw_b']
cur_df = cur_df.sort_values('x')

for i, cc in enumerate(mp_vs_sp_df['cc'].unique()):
    ax = fig.add_subplot(3, 2, i+1)
    tmp = cur_df[cur_df['cc'] == cc]
    
    ax.plot(tmp['x'], tmp['h2_tp'], label='h2 (mp)')
    ax.plot(tmp['x'], tmp['h4_tp'], label='h4 (sp)')
    ax.plot(tmp['x'], tmp['bw_b'], label='bw_b', linestyle='dashed', c='orange')
    ax.plot(tmp['x'], tmp['bw_a'], label='bw_a', linestyle='dashed', c='blue')
    ax.set_title('{}'.format(cc))
    ax.set_xlabel('bw_a / bw_b')
    ax.set_ylabel('Mbps Throughput')
    ax.set_ylim((0, cur_df['bw_b'].max()+1))
    ax.legend()
    ax.grid()
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(9.5, 9.5))
cur_df = mp_vs_sp_df[mp_vs_sp_df['bw_a'] == 15]
cur_df['x'] = cur_df['bw_a']/cur_df['bw_b']
cur_df['fair_allocation'] = cur_df[['bw_a', 'bw_b']].apply(lambda x: x[0] if x[0]>x[1] else (x[0]+x[1])/2, axis=1)
cur_df = cur_df.sort_values('x')

for i, cc in enumerate(mp_vs_sp_df['cc'].unique()):
    ax = fig.add_subplot(3, 2, i+1)
    tmp = cur_df[cur_df['cc'] == cc]
    
    ax.plot(tmp['x'], tmp['h2_tp'] / tmp['bw_a'], label='h2 (mp)')
    ax.plot(tmp['x'], tmp['h4_tp'] / tmp['bw_b'], label='h4 (sp)')
    ax.plot(tmp['x'], tmp['fair_allocation'] / tmp['bw_a'], label='fair alloc', linestyle=':')
    #ax.plot(tmp['x'], tmp['bw_b'], label='bw_b', linestyle='dashed')
    #ax.plot(tmp['x'], tmp['bw_a'], label='bw_a', linestyle='dashed')
    ax.set_title('{}'.format(cc))
    ax.set_xlabel('bw_a / bw_b')
    ax.set_ylabel('Throughput normalized')
    ax.set_ylim((0.5, 1.5))
    ax.legend()
    ax.grid()
plt.tight_layout()
# cur_df

 # Single Bottleneck

In [ ]:
single_bottleneck_df = load_iperf_experiments_new('single_bottleneck')

In [ ]:
single_bottleneck_df

In [ ]:
init_plots(single_bottleneck_df)
# de_3d_plot(single_bottleneck_df)

# Asymetric MP

In [ ]:
asym_mp_df = load_iperf_experiments_new('asym_mp')

In [ ]:
asym_mp_df['total_bw'] = 15
asym_mp_df['wasted_bw'] = asym_mp_df['total_bw'] - asym_mp_df['h2_tp'] - asym_mp_df['h4_tp'] - asym_mp_df['h6_tp']
asym_mp_df.drop(['bw_a', 'bw_b', 'de_a', 'de_b'], axis=1).set_index('cc')

In [ ]:
df = asym_mp_df.copy()
for h in ['h2', 'h4', 'h6']:
    df['diff_to_fair_' + h] = df[h + '_tp'] - 5
# asym_mp_df
df = df.drop([c for c in asym_mp_df.columns if
              not ('tp' in c or 'cc' in c or 'wasted' in c)], axis=1)


# Plot bandwidth allocation per flow
yerr = df.set_index('cc')[['h2_tp_conf', 'h4_tp_conf', 'h6_tp_conf']]
ax = df.set_index('cc')[['h2_tp', 'h4_tp', 'h6_tp']].plot.barh(stacked=True)
ax.axvline(5)
ax.axvline(10)
ax.axvline(15)

ax.set_title('Throughput per Flow')
ax.set_xlabel('Mbps')
ax.set_ylabel('')
ax.set_xlim((0, 15.5))
ax.legend()
ax.grid()
plt.tight_layout()

df